In [1]:
from PIL import Image
from PIL import ImageDraw
import random

# 1. 定义迷宫基本变量(长、宽)

In [2]:
WIDTH = 5
HEIGHT = 5
FRAME_WIDTH = WIDTH * 48 + 16
FRAME_HEIGHT = HEIGHT * 48 + 16
WALL_COLOR = (138, 111, 48)
FLOOR_COLOR = (251, 242, 54)
MIST_FLOOR_COLOR = (135, 135, 135)
FRAMES = []

# 2. 定义迷宫单元

In [3]:
class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.passed = False

# 3. 定义获取迷宫相邻方块方法

In [4]:
def getNeighborWalls(floor, maze):
    result = []
    if floor.x % 1 == 0 and floor.y % 1 == 0:
        if floor.x + 1 < len(maze):
            result.append(maze[floor.x + 1][floor.y])
        if floor.y + 1 < len(maze[floor.x]):
            result.append(maze[floor.x][floor.y + 1])
        if floor.x - 1 >= 0:
            result.append(maze[floor.x - 1][floor.y])
        if floor.y - 1 >= 0:
            result.append(maze[floor.x][floor.y - 1])
    return result


def getNeighborFloors(wall, maze):
    result = []
    if wall.x % 2 == 1 and wall.y % 2 == 0:
        if wall.y - 1 >= 0:
            result.append(maze[wall.x][wall.y - 1])
        if wall.y + 1 < len(maze[wall.x]):
            result.append(maze[wall.x][wall.y + 1])
    if wall.x % 2 == 0 and wall.y % 2 == 1:  
        if wall.x + 1 < len(maze):
            result.append(maze[wall.x + 1][wall.y])
        if wall.x - 1 >= 0:
            result.append(maze[wall.x - 1][wall.y])
    return result

# 4. 定义输出迷宫图像帧方法

In [5]:
def appendMazeFrame(maze):
    image = Image.new('RGB', (FRAME_WIDTH, FRAME_HEIGHT), color=MIST_FLOOR_COLOR)
    imaged = ImageDraw.Draw(image)
    for row in maze:
        for cell in row:
            draw_x = cell.x // 2 * 48
            draw_y = cell.y // 2 * 48
            if cell.x % 2 == 0 and cell.y % 2 == 0:
                # 死墙
                imaged.rectangle([(draw_x, draw_y), (draw_x + 16, draw_y + 16)], fill=WALL_COLOR)
            if cell.x % 2 == 0 and cell.y % 2 == 1:
                # 竖墙
                if cell.passed:
                    imaged.rectangle([(draw_x, draw_y + 16), (draw_x + 16, draw_y + 48)], fill=FLOOR_COLOR)
                else:
                    imaged.rectangle([(draw_x, draw_y + 16), (draw_x + 16, draw_y + 48)], fill=WALL_COLOR)
            if cell.x % 2 == 1 and cell.y % 2 == 0:
                # 横墙
                if cell.passed:
                    imaged.rectangle([(draw_x + 16, draw_y), (draw_x + 48, draw_y + 16)], fill=FLOOR_COLOR)
                else:
                    imaged.rectangle([(draw_x + 16, draw_y), (draw_x + 48, draw_y + 16)], fill=WALL_COLOR)
            if cell.x % 2 == 1 and cell.y % 2 == 1:
                # 地板
                if cell.passed:
                    imaged.rectangle([(draw_x + 16, draw_y + 16), (draw_x + 48, draw_y + 48)], fill=FLOOR_COLOR)
                else:
                    imaged.rectangle([(draw_x + 16, draw_y + 16), (draw_x + 48, draw_y + 48)], fill=MIST_FLOOR_COLOR)
    FRAMES.append(image)

# 5. 开始生成迷宫

In [7]:
maze = [[Cell(x, y) for y in range(HEIGHT * 2 + 1)] for x in range(WIDTH * 2 + 1)]
reached_floor = []
walls = []
step = 0
# 随机选择一块地板标记为passed，并将其相邻的墙放入判定列表
start_point = maze[2 * random.randint(0, WIDTH-1) + 1][2 * random.randint(0, HEIGHT-1) + 1]
start_point.passed = True
reached_floor.append(start_point)
walls.extend(getNeighborWalls(start_point, maze))
appendMazeFrame(maze)
step += 1
while len(walls) > 0:
    wall = walls[random.randint(0, len(walls) - 1)]
    walls.remove(wall)
    for floor in getNeighborFloors(wall, maze):
        if not floor.passed:
            wall.passed = True
            floor.passed = True
            walls.extend(getNeighborWalls(floor, maze))
            appendMazeFrame(maze)
            step += 1
appendMazeFrame(maze)

# 6. 整合图片，生成Gif展示过程

In [9]:
FRAMES[0].save('data/Prime2Gif/generating.gif', format='GIF', append_images=FRAMES[1:], save_all=True, duration=500, loop=True)